In [1]:
import numpy as np
import pandas as pd


In [2]:
movies_df = pd.read_csv("./Datasets/movies.csv" , usecols = ['movieId' , 'title'] , dtype = {'movieId' : 'int32' , 'title' : 'str'})
rating_df = pd.read_csv("./Datasets/ratings.csv" , usecols = ['userId' , 'movieId' , 'rating'] ,dtype = {'userId' : 'int32' , 'movieId' : 'int32' , 'rating' : 'float32'})

In [3]:
movies_df.head()


,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
## merging both the dataframes

df = pd.merge(rating_df , movies_df , on = 'movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [6]:
df.describe()

,userId,movieId,rating
count,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557
std,182.618491,35530.987199,1.042562
min,1.000000,1.000000,0.500000
25%,177.000000,1199.000000,3.000000
50%,325.000000,2991.000000,3.500000
75%,477.000000,8122.000000,4.000000
max,610.000000,193609.000000,5.000000


In [7]:
## dropping the null values

df = df.dropna(axis = 0, subset = ['title'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int32  
 1   movieId  100836 non-null  int32  
 2   rating   100836 non-null  float32
 3   title    100836 non-null  object 
dtypes: float32(1), int32(2), object(1)
memory usage: 2.7+ MB


In [9]:
## finding the total number of ratings for each movie

movie_ratingcount = (df. 
                     groupby(by = ['title'])['rating'].count()
                     .reset_index().rename(columns = {'rating':'totalRatingCount' })
                     [['title' , 'totalRatingCount']]
                    )

In [10]:
movie_ratingcount

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [11]:
rating_with_total_rating_count = df.merge(movie_ratingcount , left_on = 'title' , right_on = 'title' , how = 'left' '')

In [12]:
rating_with_total_rating_count

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215
...,...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997),1
100832,610,160527,4.5,Sympathy for the Underdog (1971),1
100833,610,160836,3.0,Hazard (2005),1
100834,610,163937,3.5,Blair Witch (2016),1


In [13]:
## segregating the moves based on popularity threshold

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
# plt.bar(x = rating_with_total_rating_count['totalRatingCount'] , height = 10)

In [16]:
popularity_threshold = 50
rating_popular_movie = rating_with_total_rating_count.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215
...,...,...,...,...,...
79246,603,1997,4.0,"Exorcist, The (1973)",53
79247,606,1997,3.0,"Exorcist, The (1973)",53
79248,607,1997,5.0,"Exorcist, The (1973)",53
79249,608,1997,4.5,"Exorcist, The (1973)",53


In [17]:
rating_popular_movie.shape

(41362, 5)

In [18]:
## Creating a pivot table

In [19]:
movie_features_df = rating_popular_movie.pivot_table(index = 'title' , columns = 'userId' , values = 'rating').fillna(0)
movie_features_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X2: X-Men United (2003),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
You've Got Mail (1998),0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
Young Frankenstein (1974),5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0


In [20]:
## converting it into an array matrix


In [21]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

In [22]:
movie_features_df_matrix

<450x606 sparse matrix of type '<class 'numpy.float32'>'
	with 41360 stored elements in Compressed Sparse Row format>

In [24]:
## preparing the K-nearest neighbours model

from sklearn.neighbors import NearestNeighbors


In [27]:
model_knn = NearestNeighbors(metric = 'cosine' , algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [37]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)

distances , indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1 , -1) , n_neighbors = 6)

192


In [38]:
for i in range(0 , len(distances.flatten())):
    if i ==0:
        print("The recommendation for: " , movie_features_df.index[query_index])
    else:
        print("\n" , movie_features_df.index[indices.flatten()[i]] , distances.flatten()[i])
        

The recommendation for:  Harry Potter and the Chamber of Secrets (2002)

 Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001) 0.22043866

 Harry Potter and the Prisoner of Azkaban (2004) 0.23078829

 Harry Potter and the Goblet of Fire (2005) 0.28429365

 Harry Potter and the Order of the Phoenix (2007) 0.32900852

 Pirates of the Caribbean: Dead Man's Chest (2006) 0.4141351
